In [ ]:
# for colab
# !pip install -qq rasterio timm
# !pip install -Uqq fastbook

In [ ]:
from fastai.vision.all import *
from fastai.torch_core import TensorMask, TensorImage
import rasterio as rio
import timm
import torch
import fastai
from rasterio.enums import Resampling
import torch.multiprocessing
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
model_version = "1.27"  # Assigns string value to model_version

In [ ]:
# download training data is needed

# import gdown
# import zipfile

# training_data_zip_path = Path.cwd() / "training data/training data.zip"
# training_data_zip_path.parent.mkdir(parents=True, exist_ok=True)
# url = "https://drive.google.com/uc?id=1ohuFbG7zV73ZUTIM3ybAlOk87n-nppov"
# gdown.download(url, str(training_data_zip_path), quiet=False)

# with zipfile.ZipFile(training_data_zip_path, "r") as zip_ref:
#     zip_ref.extractall(path=training_data_zip_path.parent)

In [ ]:
# for colab
# path = training_data_zip_path.parent

# for local
path = Path("/media/nick/SNEAKERNET/training data")

label_path = path / "labels_2_3_4_8_V3"

images_path = path / "images_2_3_4_8_V3"
print(label_path.exists(), images_path.exists())
print(label_path)

In [ ]:
print(torch.__version__)
print(fastai.__version__)
print(default_device())

In [ ]:
# enabling this will speed up training but can result in NAN loss
fp_16 = True

In [ ]:
# list all models available in timm
# timm.list_models()

In [ ]:
model_type = "regnety_002"

In [ ]:
model_name = (
    f"{model_type}_v{model_version}_model"  # Assigns string value to model_name
)
temp_file_name = model_name + "_temp"
model_name, temp_file_name

In [ ]:
b, g, r, nir = 1, 2, 3, 4
# how many time steps to use max is 6
time_steps = 6
# pick the bands you want to use
only_use = [g, nir]
bands_per_timestep = len(only_use)

In [ ]:
limited_band_read_list = []
for i in range(time_steps):
    for band in only_use:
        limited_band_read_list.append(i * 4 + band)
limited_band_read_list

In [ ]:
def label_func(label_path, file_path):
    label_path = label_path / file_path.name

    return label_path

In [ ]:
# func to get img list to use excluding the bad images
def get_image_files_custom(source, p=False):
    return list(source.glob("[!.]*.tif"))

In [ ]:
f_names = list(get_image_files_custom(images_path))
len(f_names)

In [ ]:
f_names[0]

In [ ]:
validation_paths = []
for img in f_names:
    if "Validation" in img.name:
        validation_paths.append(img)

len(validation_paths)

In [ ]:
def is_valid_file(x, validation_paths):
    return x in validation_paths

In [ ]:
class Switcheroo(RandTransform):
    "Randomly switch the order of the time steps, keeping the band order the same"

    split_idx, order = 0, 2

    def __init__(self, p=1, bands_per_timestep=2, time_steps=3):
        super().__init__(p=p)
        self.bands_per_timestep = bands_per_timestep
        self.time_steps = time_steps

    def encodes(self, x: (TensorImage)):
        new_time_step_order = torch.randperm(self.time_steps)
        new_order = [
            (i * self.bands_per_timestep) + j
            for i in new_time_step_order
            for j in range(self.bands_per_timestep)
        ]
        return x[:, new_order]

In [ ]:
class BatchRot90(RandTransform):
    "Rotate image and mask by 0, 90, 180, or 270 degrees"

    split_idx, order = 0, 2

    def __init__(self, p=1):
        super().__init__(p=p)
        self.rots = 0

    def before_call(self, b, split_idx):
        if random.random() < self.p:
            self.rot = random.choice([0, 1, 2, 3])
        else:
            self.rot = 0

    def encodes(self, x: (TensorImage, TensorMask)):
        return x.rot90(self.rot, [-2, -1])

In [ ]:
def open_mask(img_path, img_size):
    with rio.open(img_path) as src:
        raw_bands = src.read(
            1, out_shape=(img_size, img_size), resampling=Resampling.nearest
        )
    return TensorMask(torch.from_numpy(raw_bands))

In [ ]:
image_cache = {}

In [ ]:
def open_img(img_path, img_size):
    if img_path in image_cache:
        tensor_img = image_cache[img_path]
        if tensor_img.shape[-1] == img_size:
            return tensor_img
    with rio.open(img_path) as src:
        raw_bands = src.read(limited_band_read_list, out_shape=(img_size, img_size))

    tensor_img = TensorImage(torch.from_numpy(raw_bands.astype("float16")))
    if fp_16:
        tensor_img = tensor_img.half()

    tensor_img = tensor_img.cuda()

    image_cache[img_path] = tensor_img
    return tensor_img

In [ ]:
open_img(f_names[0], 256).shape

In [ ]:
all_means = []
all_stds = []
for i in tqdm(f_names):
    image_tensor = open_img(i, 256) / 32767
    all_means.append(image_tensor.mean((1, 2)).tolist())
    all_stds.append(image_tensor.std((1, 2)).tolist())
all_stds = np.array(all_stds).mean((0))
all_means = np.array(all_means).mean((0))
# break

In [ ]:
all_means

In [ ]:
all_stds

In [ ]:
# build datablock
def build_dblock(img_size):
    open_img_partial = partial(open_img, img_size=img_size)
    open_mask_partial = partial(open_mask, img_size=img_size)
    lable_func_partial = partial(label_func, label_path)

    label_block = TransformBlock(open_mask_partial)
    dblock = DataBlock(
        blocks=(TransformBlock(open_img_partial), label_block),
        get_items=get_image_files_custom,
        get_y=lable_func_partial,
        splitter=FuncSplitter(lambda x: is_valid_file(x, validation_paths)),
        batch_tfms=[
            IntToFloatTensor(32767, 1),
            *aug_transforms(
                flip_vert=True,
                max_rotate=0,
                max_zoom=0.2,
                max_lighting=0.2,
                max_warp=0,
                p_affine=0,
                p_lighting=0.2,
                size=img_size,
            ),
            BatchRot90(),
            # Switcheroo(bands_per_timestep=bands_per_timestep, time_steps=time_steps),
            Normalize.from_stats(mean=all_means, std=all_stds),
        ],
    )
    return dblock

In [ ]:
dl = build_dblock(256).dataloaders(images_path, bs=16, num_workers=0)

In [ ]:
ob = dl.one_batch()

In [ ]:
ob[0].shape

In [ ]:
timm_model = partial(
    timm.create_model,
    model_type,
    pretrained=True,
    in_chans=dl.one_batch()[0].shape[1],
)

In [ ]:
loss_func = MSELossFlat()
n_out = 1

In [ ]:
learner = unet_learner(
    dl, timm_model, pretrained=True, loss_func=loss_func, n_out=n_out
)
if fp_16:
    print("Converting to FP16")
    learner.to_fp16()

In [ ]:
cbs = [
    SaveModelCallback(monitor="valid_loss", fname=model_name, with_opt=True),
    ShowGraphCallback(),
]

In [ ]:
learner.fine_tune(
    freeze_epochs=10,
    epochs=30,
    cbs=cbs,
)

In [ ]:
# reload best model
learner.load(model_name)

In [ ]:
img_numb = 1
p = learner.predict(validation_paths[img_numb], with_input=True)

fig, axes = plt.subplots(1, 3, figsize=(15, 7))  # 1 row, 3 columns
axes[0].imshow(p[0].numpy()[2])
axes[0].axis("off")
axes[1].imshow(p[1].numpy()[0] > 0)
axes[1].axis("off")
axes[2].imshow(p[1].numpy()[0])
axes[2].axis("off")
plt.tight_layout()

In [ ]:
pickle.dump(learner.model, open(Path.cwd() / f"models/{model_name}.pkl", "wb"))

In [ ]:
f"{model_name}.pkl"